## Service Setup

This notebooks sets up and verifies the services needed for the Document Intelligence app. The services include:

1. Storage Service (Volumes)
2. Database Service (Lakebase)
3. Document Service (DBSQL)
4. Agent Service (Model Serving)

These services are orchestrated by the application, but this notebook sets up and verifies each service individually.

We use the workspace client extensively throughout the app and repo, so first thing is to validate our connection.

In [ ]:
# Validate Databricks connection
from doc_intel.utils import get_workspace_client
client = get_workspace_client()
client.current_user.me()

User(active=True, display_name='Scott McKean', emails=[ComplexValue(display=None, primary=True, ref=None, type='work', value='scott.mckean@databricks.com')], entitlements=[], external_id='87c2d573-1242-4b6a-8679-2443d3f8ea3e', groups=[ComplexValue(display='account users', primary=None, ref='Groups/5005646110826197', type='direct', value='5005646110826197'), ComplexValue(display='team.fieldeng-all', primary=None, ref='Groups/523217415113412', type='direct', value='523217415113412'), ComplexValue(display='admins', primary=None, ref='Groups/8460646211285870', type='direct', value='8460646211285870'), ComplexValue(display='Canada SA', primary=None, ref='Groups/502866645381412', type='direct', value='502866645381412')], id='7873535765378608', name=Name(family_name='McKean', given_name='Scott'), roles=[], schemas=[<UserSchema.URN_IETF_PARAMS_SCIM_SCHEMAS_CORE_2_0_USER: 'urn:ietf:params:scim:schemas:core:2.0:User'>, <UserSchema.URN_IETF_PARAMS_SCIM_SCHEMAS_EXTENSION_WORKSPACE_2_0_USER: 'urn:i